In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlalchemy
import pandasql as ps
from pandasql import sqldf 
mysql = lambda q: sqldf(q, globals())

In [2]:
dim_match_summary = pd.read_csv('dim_match_summary.csv')
print(dim_match_summary.info())
dim_match_summary.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45 entries, 0 to 44
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   team1      45 non-null     object
 1   team2      45 non-null     object
 2   winner     45 non-null     object
 3   margin     41 non-null     object
 4   ground     45 non-null     object
 5   matchDate  45 non-null     object
 6   match_id   45 non-null     object
dtypes: object(7)
memory usage: 2.6+ KB
None


,team1,team2,winner,margin,ground,matchDate,match_id
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,16-Oct-22,T20I # 1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,16-Oct-22,T20I # 1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,17-Oct-22,T20I # 1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,17-Oct-22,T20I # 1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,18-Oct-22,T20I # 1830


In [3]:
#Converting the matchdate column into Date format 
dim_match_summary['matchDate'] = pd.to_datetime(dim_match_summary['matchDate'], format='%d-%b-%y')

#Removing T20I# from the matchId
dim_match_summary['matchId'] = dim_match_summary['match_id'].str.extract(r'#\s(\d+[a-zA-Z]*)')

#Replacing 'no result' and 'abandoned' with 'no result' in the winner column
dim_match_summary.loc[dim_match_summary['winner'].isin(['no result', 'abandoned']), 'winner'] = 'no result'

#Removing extra spaces and converting into lower
dim_match_summary['team1'] =dim_match_summary['team1'].astype(str).str.strip().str.lower()
dim_match_summary['team2'] =dim_match_summary['team2'].astype(str).str.strip().str.lower()
dim_match_summary['winner'] =dim_match_summary['winner'].astype(str).str.strip().str.lower()
dim_match_summary['ground'] =dim_match_summary['ground'].astype(str).str.strip().str.lower()


# Converting  teamInnings and batsmanName to Proper case
dim_match_summary['team1'] =dim_match_summary['team1'].str.title()
dim_match_summary['team2'] =dim_match_summary['team2'].str.title()
dim_match_summary['winner'] =dim_match_summary['winner'].str.title()
dim_match_summary['ground'] =dim_match_summary['ground'].str.title()


# Set margin to null for no result  matches
dim_match_summary.loc[dim_match_summary['winner'] == 'no result', 'margin'] = None

#Checking consistency
same_team_check = dim_match_summary[dim_match_summary['team1'] == dim_match_summary['team2']]

# Display rows where team1 and team2 are the same
if same_team_check.empty:
    print("All rows have different teams for 'team1' and 'team2'.")
else:
    print("There are rows where 'team1' and 'team2' are the same:")
    print(same_team_check)

print(dim_match_summary.duplicated().sum())
dim_match_summary.drop(columns='match_id',inplace=True)
dim_match_summary.to_csv('match_summary.csv', index=False)
dim_match_summary

All rows have different teams for 'team1' and 'team2'.
0


,team1,team2,winner,margin,ground,matchDate,matchId
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,2022-10-16,1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,2022-10-16,1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,2022-10-17,1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,2022-10-17,1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,2022-10-18,1830
5,Sri Lanka,U.A.E.,Sri Lanka,79 runs,Geelong,2022-10-18,1832
6,Ireland,Scotland,Ireland,6 wickets,Hobart,2022-10-19,1833
7,West Indies,Zimbabwe,West Indies,31 runs,Hobart,2022-10-19,1834
8,Netherlands,Sri Lanka,Sri Lanka,16 runs,Geelong,2022-10-20,1835
9,Namibia,U.A.E.,U.A.E.,7 runs,Geelong,2022-10-20,1836


In [4]:
dim_players = pd.read_csv('dim_players.csv')
print(dim_players.info())
dim_players.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 219 entries, 0 to 218
Data columns (total 5 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   name          219 non-null    object
 1   team          219 non-null    object
 2   battingStyle  219 non-null    object
 3   bowlingStyle  199 non-null    object
 4   playingRole   219 non-null    object
dtypes: object(5)
memory usage: 8.7+ KB
None


,name,team,battingStyle,bowlingStyle,playingRole
0,Najmul Hossain Shanto,Bangladesh,Left hand Bat,Right arm Offbreak,Top order Batter
1,Soumya Sarkar,Bangladesh,Left hand Bat,Right arm Medium fast,Middle order Batter
2,Litton Das,Bangladesh,Right hand Bat,NaN,Wicketkeeper Batter
3,Shakib Al Hasan(c),Bangladesh,Left hand Bat,Slow Left arm Orthodox,Allrounder
4,Afif Hossain,Bangladesh,Left hand Bat,Right arm Offbreak,Allrounder


In [5]:
 
import pandas as pd


dim_players = pd.read_csv('dim_players.csv')

# Filling NaN values with specified defaults
dim_players['bowlingStyle'].fillna('None', inplace=True)
dim_players['battingStyle'].fillna('None', inplace=True)
dim_players['playingRole'].fillna('Unknown', inplace=True)

# Removing (C) from the 'name' column 
dim_players['name'] = dim_players['name'].str.replace(r'\(C\)', '', regex=True).str.strip()

# Converting  teamInnings and batsmanName to Proper case
dim_players['name'] = dim_players['name'].astype(str).str.strip().str.lower().str.title()
dim_players['playingRole'] = dim_players['playingRole'].str.strip().str.lower().str.title()
dim_players['team'] = dim_players['team'].str.strip().str.lower().str.title()
dim_players['battingStyle'] = dim_players['battingStyle'].str.strip().str.lower().str.title()
dim_players['bowlingStyle'] = dim_players['bowlingStyle'].str.strip().str.lower().str.title()
dim_players['name'] = dim_players['name'].str.replace(r'\(C\)', '', regex=True).str.strip()


# Checking for duplicates in the DataFrame
print(f"Number of duplicate rows: {dim_players.duplicated().sum()}")


dim_players.to_csv('Infoplayers.csv', index=False)


dim_players.head(30)


Number of duplicate rows: 6


,name,team,battingStyle,bowlingStyle,playingRole
0,Najmul Hossain Shanto,Bangladesh,Left Hand Bat,Right Arm Offbreak,Top Order Batter
1,Soumya Sarkar,Bangladesh,Left Hand Bat,Right Arm Medium Fast,Middle Order Batter
2,Litton Das,Bangladesh,Right Hand Bat,None,Wicketkeeper Batter
3,Shakib Al Hasan,Bangladesh,Left Hand Bat,Slow Left Arm Orthodox,Allrounder
4,Afif Hossain,Bangladesh,Left Hand Bat,Right Arm Offbreak,Allrounder
5,Mosaddek Hossain,Bangladesh,Right Hand Bat,Right Arm Offbreak,Middle Order Batter
6,Nurul Hasan,Bangladesh,Right Hand Bat,None,Wicketkeeper Batter
7,Yasir Ali,Bangladesh,Right Hand Bat,Right Arm Offbreak,Middle Order Batter
8,Wessly Madhevere,Zimbabwe,Right Hand Bat,Right Arm Offbreak,Allrounder
9,Craig Ervine,Zimbabwe,Left Hand Bat,Right Arm Offbreak,Middle Order Batter


In [6]:
fact_bowling_summary = pd.read_csv('fact_bowling_summary.csv')
print(fact_bowling_summary.info())
fact_bowling_summary.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 14 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   match        500 non-null    object 
 1   bowlingTeam  500 non-null    object 
 2   bowlerName   500 non-null    object 
 3   overs        500 non-null    float64
 4   maiden       500 non-null    int64  
 5   runs         500 non-null    int64  
 6   wickets      500 non-null    int64  
 7   economy      500 non-null    float64
 8   0s           500 non-null    int64  
 9   4s           500 non-null    int64  
 10  6s           500 non-null    int64  
 11  wides        500 non-null    int64  
 12  noBalls      500 non-null    int64  
 13  match_id     500 non-null    object 
dtypes: float64(2), int64(8), object(4)
memory usage: 54.8+ KB
None


,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,match_id
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4.0,0,23,1,5.75,7,0,0,2,0,T20I # 1823
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4.0,0,39,1,9.75,6,3,1,2,0,T20I # 1823
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4.0,0,37,2,9.25,6,3,1,0,0,T20I # 1823
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4.0,0,36,1,9.00,7,3,1,1,0,T20I # 1823
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga de Silva,4.0,0,27,1,6.75,8,1,1,0,0,T20I # 1823


In [7]:
#Removing T20I# from the matchId
fact_bowling_summary['matchId'] = fact_bowling_summary['match_id'].str.extract(r'#\s(\d+[a-zA-Z]*)')

#Calculating Overall Extras Conceded
fact_bowling_summary['extras'] = fact_bowling_summary['wides'] + fact_bowling_summary['noBalls']

#Removing extra spaces and converting into lower
fact_bowling_summary['bowlingTeam'] = fact_bowling_summary['bowlingTeam'].astype(str).str.strip().str.lower()
fact_bowling_summary['bowlerName'] = fact_bowling_summary['bowlerName'].astype(str).str.strip().str.lower()

#Converting  teamInnings and batsmanName to Proper case
fact_bowling_summary['bowlingTeam'] = fact_bowling_summary['bowlingTeam'].str.title()
fact_bowling_summary['bowlerName'] =  fact_bowling_summary['bowlerName'].str.title()
fact_bowling_summary['bowlerName'] = fact_bowling_summary['bowlerName'].str.replace(r'\(C\)', '', regex=True).str.strip()


#Detecting outliers in the economy
outliers = fact_bowling_summary[
    (fact_bowling_summary['economy'] > 36) |
    (fact_bowling_summary['overs'] > 4.0) |
    (fact_bowling_summary['wickets'] > 10) |
    (fact_bowling_summary['maiden'] > 4)
]

if outliers.empty:
    print("No Oultiers")
else:
    print("Outliers Detected:\n", outliers)
q="""SELECT bowlerName,runs,economy,
    (runs / economy) AS actual_overs_bowled
FROM
    outliers;
"""
actual_overs=mysql(q)
actual_overs
outliers


Outliers Detected:
                     match  bowlingTeam   bowlerName  overs  maiden  runs  \
12  U.A.E. Vs Netherlands  Netherlands  Tim Pringle    5.0       0    13   

    wickets  economy  0s  4s  6s  wides  noBalls       match_id matchId  \
12        1     3.25  11   0   0      0        0  T20I # 1825      1825   

    extras  
12       0  


,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,match_id,matchId,extras
12,U.A.E. Vs Netherlands,Netherlands,Tim Pringle,5.0,0,13,1,3.25,11,0,0,0,0,T20I # 1825,1825,0


In [8]:
fact_bowling_summary.loc[
    (fact_bowling_summary['matchId'] == '1825') & 
    (fact_bowling_summary['bowlerName'] == 'Tim Pringle'), 
    'overs'
] =actual_overs['actual_overs_bowled'][0]
updated_row = fact_bowling_summary[
    (fact_bowling_summary['matchId'] == '1825') & 
    (fact_bowling_summary['bowlerName'] == 'Tim Pringle')
]
updated_row

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,match_id,matchId,extras
12,U.A.E. Vs Netherlands,Netherlands,Tim Pringle,4.0,0,13,1,3.25,11,0,0,0,0,T20I # 1825,1825,0


In [9]:
#Calculating Number of balls bowled
fact_bowling_summary['total_balls_bowled'] = (fact_bowling_summary['overs'].astype(int) * 6) + ((fact_bowling_summary['overs'] % 1) * 10).astype(int)

print(f"Number of duplicate rows: {fact_bowling_summary.duplicated().sum()}")
fact_bowling_summary.drop(columns='match_id',inplace=True)
fact_bowling_summary.to_csv('bowling_summary.csv', index=False)

fact_bowling_summary

Number of duplicate rows: 0


,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,matchId,extras,total_balls_bowled
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4.0,0,23,1,5.75,7,0,0,2,0,1823,2,24
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4.0,0,39,1,9.75,6,3,1,2,0,1823,2,24
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4.0,0,37,2,9.25,6,3,1,0,0,1823,0,24
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4.0,0,36,1,9.00,7,3,1,1,0,1823,1,24
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga De Silva,4.0,0,27,1,6.75,8,1,1,0,0,1823,0,24
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,Pakistan Vs England,Pakistan,Naseem Shah,4.0,0,30,0,7.50,15,3,1,1,0,1879,1,24
496,Pakistan Vs England,Pakistan,Haris Rauf,4.0,0,23,2,5.75,13,3,0,1,0,1879,1,24
497,Pakistan Vs England,Pakistan,Shadab Khan,4.0,0,20,1,5.00,10,1,0,0,0,1879,0,24
498,Pakistan Vs England,Pakistan,Mohammad Wasim,4.0,0,38,1,9.50,5,5,0,2,0,1879,2,24


In [10]:
fact_batting_summary = pd.read_csv('fact_bating_summary.csv')
print(fact_batting_summary.info())
fact_batting_summary.head()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 699 entries, 0 to 698
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   match        699 non-null    object
 1   teamInnings  699 non-null    object
 2   battingPos   699 non-null    int64 
 3   batsmanName  699 non-null    object
 4   runs         699 non-null    int64 
 5   balls        699 non-null    int64 
 6   4s           699 non-null    int64 
 7   6s           699 non-null    int64 
 8   SR           699 non-null    object
 9   out/not_out  699 non-null    object
 10  match_id     699 non-null    object
dtypes: int64(5), object(6)
memory usage: 60.2+ KB
None


,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/not_out,match_id
0,Namibia Vs Sri Lanka,Namibia,1,Michael van Lingen,3,6,0,0,50,out,T20I # 1823
1,Namibia Vs Sri Lanka,Namibia,2,Divan la Cock,9,9,1,0,100,out,T20I # 1823
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,out,T20I # 1823
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,out,T20I # 1823
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus(c),20,24,0,0,83.33,out,T20I # 1823


In [11]:
#Removing T20I# from the matchId
fact_batting_summary['matchId'] = fact_batting_summary['match_id'].str.extract(r'#\s(\d+[a-zA-Z]*)')

#Converting SR to Float Type
fact_batting_summary['SR'] = fact_batting_summary['SR'].replace({'-': np.nan}).fillna(0.00).astype(float).round(2)


#Removing extra spaces and converting into lower
fact_batting_summary['teamInnings'] = fact_batting_summary['teamInnings'].astype(str).str.strip().str.lower()
fact_batting_summary['batsmanName'] = fact_batting_summary['batsmanName'].astype(str).str.strip().str.lower()

# Converting  teamInnings and batsmanName to Proper case
fact_batting_summary['teamInnings'] = fact_batting_summary['teamInnings'].str.title()
fact_batting_summary['batsmanName'] = fact_batting_summary['batsmanName'].str.title()


# Converting out/not_out to binary
print(fact_batting_summary['out/not_out'].unique())
fact_batting_summary['out/not_out'] = fact_batting_summary['out/not_out'].astype(str).str.strip().str.lower()
fact_batting_summary['out/not_out'] = fact_batting_summary['out/not_out'].map({'out': 1, 'not_out': 0})
fact_batting_summary['batsmanName'] = fact_batting_summary['batsmanName'].str.replace(r'\(C\)', '', regex=True).str.strip()


# Checking Outliers
normal_outliers = fact_batting_summary[
    (fact_batting_summary['runs'] < 0) |
    (fact_batting_summary['balls'] < 0) |
    (fact_batting_summary['4s'] < 0) |(fact_batting_summary['4s'] > fact_batting_summary['balls']) |
    (fact_batting_summary['6s'] < 0) |(fact_batting_summary['6s'] > fact_batting_summary['balls']) |
    (fact_batting_summary['SR'] < 0)
]
if normal_outliers.empty:
    print("No  Normal Oultiers")
else:
    print("Outliers Detected:\n",normal_outliers)


#Checking SR outliers
fact_batting_summary['calculated_SR'] = np.where(
    fact_batting_summary['balls'] > 0,
    (fact_batting_summary['runs'] / fact_batting_summary['balls']) * 100,
    0
)
strike_rate_outlier = fact_batting_summary[
    (fact_batting_summary['balls'] > 0) &  
    (abs(fact_batting_summary['SR'] - fact_batting_summary['calculated_SR']) > 1)
]
print(strike_rate_outlier)
q="""SELECT *,
    ROUND((SR * balls) / 100)  AS calculated_runs
FROM strike_rate_outlier;
"""
SR_outlier=mysql(q)
SR_outlier




['out' 'not_out' 'Out' 'not_Out' 'out  ' 'not_out  ']
No  Normal Oultiers
                      match teamInnings  battingPos       batsmanName  runs  \
263       Pakistan Vs India       India           5        Axar Patel    62   
407  Bangladesh Vs Zimbabwe    Zimbabwe           1  Wessly Madhevere    74   

     balls  4s  6s      SR  out/not_out     match_id matchId  calculated_SR  
263      3   0   0   66.66            1  T20I # 1842    1842    2066.666667  
407      3   1   0  133.33            1  T20I # 1851    1851    2466.666667  


,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/not_out,match_id,matchId,calculated_SR,calculated_runs
0,Pakistan Vs India,India,5,Axar Patel,62,3,0,0,66.66,1,T20I # 1842,1842,2066.666667,2.0
1,Bangladesh Vs Zimbabwe,Zimbabwe,1,Wessly Madhevere,74,3,1,0,133.33,1,T20I # 1851,1851,2466.666667,4.0


In [12]:
fact_batting_summary.drop(columns=['match_id','calculated_SR'],inplace=True)
for index, row in SR_outlier.iterrows():
    fact_batting_summary.loc[
        (fact_batting_summary['matchId'] == row['matchId']) & 
        (fact_batting_summary['teamInnings'] == row['teamInnings']) & 
        (fact_batting_summary['battingPos'] == row['battingPos']) & 
        (fact_batting_summary['batsmanName'] == row['batsmanName']),
        'runs'
    ] = row['calculated_runs']

updated_rows = fact_batting_summary[
    (fact_batting_summary['matchId'].isin(SR_outlier['matchId'])) & 
    (fact_batting_summary['batsmanName'].isin(SR_outlier['batsmanName']))
]

updated_rows

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/not_out,matchId
263,Pakistan Vs India,India,5,Axar Patel,2,3,0,0,66.66,1,1842
407,Bangladesh Vs Zimbabwe,Zimbabwe,1,Wessly Madhevere,4,3,1,0,133.33,1,1851


In [13]:
fact_batting_summary.to_csv('batting_summary.csv', index=False)
fact_batting_summary

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/not_out,matchId
0,Namibia Vs Sri Lanka,Namibia,1,Michael Van Lingen,3,6,0,0,50.00,1,1823
1,Namibia Vs Sri Lanka,Namibia,2,Divan La Cock,9,9,1,0,100.00,1,1823
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,1,1823
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,1,1823
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus,20,24,0,0,83.33,1,1823
...,...,...,...,...,...,...,...,...,...,...,...
694,Pakistan Vs England,England,3,Phil Salt,10,9,2,0,111.11,1,1879
695,Pakistan Vs England,England,4,Ben Stokes,52,49,5,1,106.12,0,1879
696,Pakistan Vs England,England,5,Harry Brook,20,23,1,0,86.95,1,1879
697,Pakistan Vs England,England,6,Moeen Ali,19,13,3,0,146.15,1,1879


In [14]:
print(fact_batting_summary['out/not_out'].unique())

[1 0]


### KPI's Identification 

In [15]:
matchdf=pd.read_csv('match_summary.csv')
matchdf.head()

,team1,team2,winner,margin,ground,matchDate,matchId
0,Namibia,Sri Lanka,Namibia,55 runs,Geelong,2022-10-16,1823
1,Netherlands,U.A.E.,Netherlands,3 wickets,Geelong,2022-10-16,1825
2,Scotland,West Indies,Scotland,42 runs,Hobart,2022-10-17,1826
3,Ireland,Zimbabwe,Zimbabwe,31 runs,Hobart,2022-10-17,1828
4,Namibia,Netherlands,Netherlands,5 wickets,Geelong,2022-10-18,1830


In [16]:
battingdf=pd.read_csv('batting_summary.csv')
battingdf.head()

,match,teamInnings,battingPos,batsmanName,runs,balls,4s,6s,SR,out/not_out,matchId
0,Namibia Vs Sri Lanka,Namibia,1,Michael Van Lingen,3,6,0,0,50.00,1,1823
1,Namibia Vs Sri Lanka,Namibia,2,Divan La Cock,9,9,1,0,100.00,1,1823
2,Namibia Vs Sri Lanka,Namibia,3,Jan Nicol Loftie-Eaton,20,12,1,2,166.66,1,1823
3,Namibia Vs Sri Lanka,Namibia,4,Stephan Baard,26,24,2,0,108.33,1,1823
4,Namibia Vs Sri Lanka,Namibia,5,Gerhard Erasmus,20,24,0,0,83.33,1,1823


In [17]:
bowlingdf=pd.read_csv('bowling_summary.csv')
bowlingdf.head()

,match,bowlingTeam,bowlerName,overs,maiden,runs,wickets,economy,0s,4s,6s,wides,noBalls,matchId,extras,total_balls_bowled
0,Namibia Vs Sri Lanka,Sri Lanka,Maheesh Theekshana,4.0,0,23,1,5.75,7,0,0,2,0,1823,2,24
1,Namibia Vs Sri Lanka,Sri Lanka,Dushmantha Chameera,4.0,0,39,1,9.75,6,3,1,2,0,1823,2,24
2,Namibia Vs Sri Lanka,Sri Lanka,Pramod Madushan,4.0,0,37,2,9.25,6,3,1,0,0,1823,0,24
3,Namibia Vs Sri Lanka,Sri Lanka,Chamika Karunaratne,4.0,0,36,1,9.00,7,3,1,1,0,1823,1,24
4,Namibia Vs Sri Lanka,Sri Lanka,Wanindu Hasaranga De Silva,4.0,0,27,1,6.75,8,1,1,0,0,1823,0,24


In [18]:
playersdf=pd.read_csv('Infoplayers.csv')

playersdf.head()


,name,team,battingStyle,bowlingStyle,playingRole
0,Najmul Hossain Shanto,Bangladesh,Left Hand Bat,Right Arm Offbreak,Top Order Batter
1,Soumya Sarkar,Bangladesh,Left Hand Bat,Right Arm Medium Fast,Middle Order Batter
2,Litton Das,Bangladesh,Right Hand Bat,NaN,Wicketkeeper Batter
3,Shakib Al Hasan,Bangladesh,Left Hand Bat,Slow Left Arm Orthodox,Allrounder
4,Afif Hossain,Bangladesh,Left Hand Bat,Right Arm Offbreak,Allrounder


# Overall Tournament KPIs



- **Total Matches:** Total number of unique matches played in the tournament.
- **Total Runs Scored:** Aggregate number of runs scored by all teams across all matches.
- **Total Wickets Taken:** Total number of wickets taken by bowlers in all matches.
- **Total Boundaries:** Sum of all boundaries (fours and sixes) hit during the matches.
- **Average Runs Per Innings:** Average number of runs scored per innings.
- **Highest Run Scorer:** Player with the most runs scored across all matches.
- **Highest Wicket Taker:** Bowler with the most wickets taken across all matches.
- **Highest Individual Score:** Highest score achieved by a batsman in a single innings.
- **Highest Individual Wickets:** Highest number of wickets taken by a bowler in a single innings.


In [19]:
import pandas as pd

queries = {
    "Total Matches": """
        SELECT COUNT(DISTINCT matchId) as Total_Matches 
        FROM matchdf;
    """,
    "Total Runs Scored": """
        SELECT SUM(runs) AS total_runs_scored
        FROM battingdf;
    """,
    "Total Wickets Taken": """
        SELECT SUM(wickets) AS total_wickets_taken
        FROM bowlingdf;
    """,
    "Total Boundaries": """
        SELECT SUM("4s") + SUM("6s") AS Total_boundaries
        FROM battingdf;
    """,
    "Average Runs Per Innings": """
        SELECT ROUND(AVG(total_runs), 2) AS avg_runs_per_innings
        FROM (
            SELECT matchId, teamInnings, SUM(runs) AS total_runs
            FROM battingdf
            GROUP BY matchId, teamInnings
        ) AS innings_runs;
    """,
    "Highest Run Scorer": """
        SELECT batsmanName,
               SUM(runs) AS total_runs
        FROM battingdf
        GROUP BY batsmanName
        ORDER BY total_runs DESC
        LIMIT 1;
    """,
    "Highest Wicket Taker": """
        SELECT bowlerName,
               SUM(wickets) AS total_wickets
        FROM bowlingdf
        GROUP BY bowlerName
        ORDER BY total_wickets DESC
        LIMIT 1;
    """,
    "Highest Individual Score": """
        SELECT batsmanName, MAX(runs) AS highest_individual_score, matchId
        FROM battingdf
        GROUP BY batsmanName, matchId
        ORDER BY highest_individual_score DESC
        LIMIT 1;
    """,
    "Highest Individual Wickets": """
        SELECT bowlerName, MAX(wickets) AS highest_individual_wickets, runs AS runs_given, matchId
        FROM bowlingdf
        GROUP BY bowlerName, runs, matchId
        ORDER BY highest_individual_wickets DESC
        LIMIT 1;
    """
}

results = {}
for kpi_name, query in queries.items():
    df = mysql(query)
    if kpi_name in ["Highest Run Scorer", "Highest Wicket Taker", "Highest Individual Score", "Highest Individual Wickets"]:
        if kpi_name == "Highest Run Scorer":
            results[kpi_name] = f"{df.iloc[0]['batsmanName']} ({df.iloc[0]['total_runs']})"
        elif kpi_name == "Highest Wicket Taker":
            results[kpi_name] = f"{df.iloc[0]['bowlerName']} ({df.iloc[0]['total_wickets']})"
        elif kpi_name == "Highest Individual Score":
            results[kpi_name] = f"{df.iloc[0]['batsmanName']} ({df.iloc[0]['highest_individual_score']})"
        elif kpi_name == "Highest Individual Wickets":
            results[kpi_name] = f"{df.iloc[0]['bowlerName']} ({df.iloc[0]['highest_individual_wickets']})"
    else:
        results[kpi_name] = df.iloc[0, 0]

kpi_df = pd.DataFrame(list(results.items()), columns=["Metric", "Value"])

kpi_df


,Metric,Value
0,Total Matches,45
1,Total Runs Scored,11169
2,Total Wickets Taken,515
3,Total Boundaries,1240
4,Average Runs Per Innings,132.96
5,Highest Run Scorer,Virat Kohli (296)
6,Highest Wicket Taker,Wanindu Hasaranga De Silva (15)
7,Highest Individual Score,Rilee Rossouw (109)
8,Highest Individual Wickets,Sam Curran (5)


# Team Specific KPI's


- **Total Matches:** Total number of unique matches played by the team.
- **Win/Loss Ratio:** Percentage of matches won compared to the total number of matches played by the team.
- **Total Runs Scored:** Aggregate number of runs scored by the team across all matches.
- **Total Wickets Taken:** Total number of wickets taken by the team across all matches.
- **Total Boundaries:** Sum of all boundaries (fours and sixes) hit by the team.
- **Average Runs Per Innings:** Average number of runs scored by the team per innings.
- **Average Economy Rate:** Average number of runs conceded per over by the team’s bowlers.
- **Highest Run Scorer:** Player with the most runs scored for the team.
- **Highest Wicket Taker:** Bowler with the most wickets taken for the team.


In [20]:
import pandas as pd

team_name = 'India'

queries = {
    "Total Matches": f"""
        SELECT COUNT(DISTINCT matchId) AS Total_Matches 
        FROM matchdf
        WHERE team1 = '{team_name}' OR team2='{team_name}';
    """,
    "Win/Loss Ratio": f"""
        SELECT 
        ROUND(SUM(wins) * 1.0 / SUM(total_matches) * 100, 2) AS win_loss_ratio
        FROM (
            SELECT 
                SUM(CASE WHEN winner = team1 THEN 1 ELSE 0 END) AS wins,
                COUNT(*) AS total_matches
            FROM matchdf
            WHERE team1 = '{team_name}'
            UNION ALL
            SELECT 
                SUM(CASE WHEN winner = team2 THEN 1 ELSE 0 END) AS wins,
                COUNT(*) AS total_matches
            FROM matchdf
            WHERE team2 = '{team_name}'
        ) AS combined
    """,
    "Total Runs Scored": f"""
        SELECT SUM(runs) AS total_runs_scored
        FROM battingdf
        WHERE teamInnings = '{team_name}';
    """,
    "Total Wickets Taken": f"""
        SELECT SUM(wickets) AS total_wickets_taken
        FROM bowlingdf
        WHERE bowlingTeam = '{team_name}';
    """,
    "Total Boundaries": f"""
        SELECT SUM("4s") + SUM("6s") AS Total_boundaries
        FROM battingdf
        WHERE teamInnings = '{team_name}';
    """,
    "AVG Runs Per Innings": f"""
        SELECT ROUND(AVG(total_runs), 2) AS avg_runs_per_innings
        FROM (
            SELECT matchId, teamInnings, SUM(runs) AS total_runs
            FROM battingdf
            WHERE teamInnings = '{team_name}'
            GROUP BY matchId, teamInnings
        ) AS innings_runs;
    """,
    "AVG Economy Rate": f"""
        SELECT ROUND(SUM(runs) * 1.0 / NULLIF(SUM(overs), 0), 2) AS economy_rate
        FROM bowlingdf
        WHERE bowlingTeam = '{team_name}';
    """,
    "Highest Run Scorer": f"""
        SELECT batsmanName,
               SUM(runs) AS total_runs
        FROM battingdf
        WHERE teamInnings = '{team_name}'
        GROUP BY batsmanName
        ORDER BY total_runs DESC
        LIMIT 1;
    """,
    "Highest Wicket Taker": f"""
        SELECT bowlerName,
               SUM(wickets) AS total_wickets
        FROM bowlingdf
        WHERE bowlingTeam = '{team_name}'
        GROUP BY bowlerName
        ORDER BY total_wickets DESC
        LIMIT 1;
    """
}

results = {}
for kpi_name, query in queries.items():
    df = mysql(query)
    if kpi_name in ["Highest Run Scorer", "Highest Wicket Taker"]:
        if not df.empty:
            if kpi_name == "Highest Run Scorer":
                results[kpi_name] = f"{df.iloc[0]['batsmanName']} ({df.iloc[0]['total_runs']})"
            elif kpi_name == "Highest Wicket Taker":
                results[kpi_name] = f"{df.iloc[0]['bowlerName']} ({df.iloc[0]['total_wickets']})"
    else:
        results[kpi_name] = df.iloc[0, 0] if not df.empty else None

team_kpi = pd.DataFrame(list(results.items()), columns=["Metric", "Value"])

team_kpi


,Metric,Value
0,Total Matches,6
1,Win/Loss Ratio,66.67
2,Total Runs Scored,966
3,Total Wickets Taken,37
4,Total Boundaries,118
5,AVG Runs Per Innings,161.0
6,AVG Economy Rate,7.64
7,Highest Run Scorer,Virat Kohli (296)
8,Highest Wicket Taker,Arshdeep Singh (10)


# Player Specific KPIs

## Batter

- **Total Runs Scored:** Total number of runs scored by the player.
- **Batting Average:** Average number of runs scored per innings where the player was out.
- **Average Balls Faced:** Average number of balls faced per innings.
- **Average Strike Rate:** Average strike rate of the player.
- **Boundary Percentage:** Percentage of balls faced that resulted in boundaries (fours or sixes).
- **Boundaries Hit Per Match:** Average number of boundaries hit per match.
- **Boundary Frequency (balls per boundary):** Average number of balls faced per boundary hit.

## Bowler

- **Total Wickets Taken:** Total number of wickets taken by the player.
- **Bowling Average:** Average number of runs conceded per wicket taken.
- **Economy Rate:** Average number of runs conceded per over bowled.
- **Bowling Strike Rate:** Average number of balls bowled per wicket taken.
- **Dot Balls Bowled Percentage:** Percentage of balls bowled that resulted in dot balls.






In [47]:
def generate_player_kpis(player_name):
    queries = {
        "Batter": {
            "Total Runs Scored": """
                SELECT SUM(runs) AS total_runs_scored 
                FROM battingdf 
                WHERE batsmanName = '{player_name}'
                GROUP BY batsmanName;
            """,
            "Batting Average": """
                SELECT CAST(SUM(runs) AS FLOAT) / NULLIF(COUNT(DISTINCT CASE WHEN `out/not_out` = 1 THEN matchId END), 0) AS batting_average 
                FROM battingdf 
                WHERE batsmanName = '{player_name}'
                GROUP BY batsmanName;
            """,
            "Average Balls Faced": """
                SELECT CAST(SUM(balls) AS FLOAT) / NULLIF(COUNT(DISTINCT matchId), 0) AS avg_balls_faced 
                FROM battingdf 
                WHERE batsmanName = '{player_name}'
                GROUP BY batsmanName;
            """,
            "Average StrikeRate": """
                SELECT AVG(SR) AS average_strike_rate
                FROM battingdf
                WHERE batsmanName = '{player_name}';
            """,
            "Boundary Percentage": """
                SELECT (CAST(SUM(`4s`) + SUM(`6s`) AS FLOAT) / NULLIF(SUM(balls), 0)) * 100 AS boundary_percentage 
                FROM battingdf 
                WHERE batsmanName = '{player_name}'
                GROUP BY batsmanName;
            """,
            "Boundaries Hit Per Match": """
                SELECT (CAST(SUM(`4s`) + SUM(`6s`) AS FLOAT) / NULLIF(COUNT(DISTINCT matchId), 0)) AS boundaries_per_match 
                FROM battingdf 
                WHERE batsmanName = '{player_name}'
                GROUP BY batsmanName;
            """,
            "Boundary Frequency (balls per boundary)": """
                SELECT (SUM(balls) / NULLIF(SUM(`4s` + `6s`), 0)) AS boundary_frequency 
                FROM battingdf 
                WHERE batsmanName = '{player_name}'
                GROUP BY batsmanName;
            """
        },
        "Bowler": {
            "Total Wickets Taken": """
                SELECT SUM(wickets) AS total_wickets_taken 
                FROM bowlingdf 
                WHERE bowlerName = '{player_name}'
                GROUP BY bowlerName;
            """,
            "Bowling Average": """
                SELECT CAST(SUM(runs) AS FLOAT) / NULLIF(SUM(wickets), 0) AS bowling_average 
                FROM bowlingdf 
                WHERE bowlerName = '{player_name}'
                GROUP BY bowlerName;
            """,
            "Economy Rate": """
                SELECT CAST(SUM(runs) AS FLOAT) / NULLIF(SUM(total_balls_bowled) / 6, 0) AS economy_rate 
                FROM bowlingdf 
                WHERE bowlerName = '{player_name}'
                GROUP BY bowlerName;
            """,
            "Bowling Strike Rate": """
                SELECT CAST(SUM(total_balls_bowled) AS FLOAT) / NULLIF(SUM(wickets), 0) AS bowling_strike_rate 
                FROM bowlingdf 
                WHERE bowlerName = '{player_name}'
                GROUP BY bowlerName;
            """,
            "Dot Balls Bowled Percentage": """
                SELECT (CAST(SUM(`0s`) AS FLOAT) / CAST(SUM(total_balls_bowled) AS FLOAT)) * 100 AS dot_balls_bowled_percentage 
                FROM bowlingdf 
                WHERE bowlerName = '{player_name}'
                GROUP BY bowlerName;
            """
        },
        "Allrounder": {
            "Total Runs Scored": """
                SELECT SUM(runs) AS total_runs_scored 
                FROM battingdf 
                WHERE batsmanName = '{player_name}'
                GROUP BY batsmanName;
            """,
            "Batting Average": """
                SELECT CAST(SUM(runs) AS FLOAT) / NULLIF(COUNT(DISTINCT CASE WHEN `out/not_out` = 1 THEN matchId END), 0) AS batting_average 
                FROM battingdf 
                WHERE batsmanName = '{player_name}'
                GROUP BY batsmanName;
            """,
            "Average Balls Faced": """
                SELECT CAST(SUM(balls) AS FLOAT) / NULLIF(COUNT(DISTINCT matchId), 0) AS avg_balls_faced 
                FROM battingdf 
                WHERE batsmanName = '{player_name}'
                GROUP BY batsmanName;
            """,
            "Boundary Percentage": """
                SELECT (CAST(SUM(`4s`) + SUM(`6s`) AS FLOAT) / NULLIF(SUM(balls), 0)) * 100 AS boundary_percentage 
                FROM battingdf 
                WHERE batsmanName = '{player_name}'
                GROUP BY batsmanName;
            """,
            "Boundaries Hit Per Match": """
                SELECT (CAST(SUM(`4s`) + SUM(`6s`) AS FLOAT) / NULLIF(COUNT(DISTINCT matchId), 0)) AS boundaries_per_match 
                FROM battingdf 
                WHERE batsmanName = '{player_name}'
                GROUP BY batsmanName;
            """,
            "Boundary Frequency (balls per boundary)": """
                SELECT (SUM(balls) / NULLIF(SUM(`4s` + `6s`), 0)) AS boundary_frequency 
                FROM battingdf 
                WHERE batsmanName = '{player_name}'
                GROUP BY batsmanName;
            """,
            "Total Wickets Taken": """
                SELECT SUM(wickets) AS total_wickets_taken 
                FROM bowlingdf 
                WHERE bowlerName = '{player_name}'
                GROUP BY bowlerName;
            """,
            "Bowling Average": """
                SELECT CAST(SUM(runs) AS FLOAT) / NULLIF(SUM(wickets), 0) AS bowling_average 
                FROM bowlingdf 
                WHERE bowlerName = '{player_name}'
                GROUP BY bowlerName;
            """,
            "Economy Rate": """
                SELECT CAST(SUM(runs) AS FLOAT) / NULLIF(SUM(total_balls_bowled) / 6, 0) AS economy_rate 
                FROM bowlingdf 
                WHERE bowlerName = '{player_name}'
                GROUP BY bowlerName;
            """,
            "Bowling Strike Rate": """
                SELECT CAST(SUM(total_balls_bowled) AS FLOAT) / NULLIF(SUM(wickets), 0) AS bowling_strike_rate 
                FROM bowlingdf 
                WHERE bowlerName = '{player_name}'
                GROUP BY bowlerName;
            """,
            "Dot Balls Bowled Percentage": """
                SELECT (CAST(SUM(`0s`) AS FLOAT) / CAST(SUM(total_balls_bowled) AS FLOAT)) * 100 AS dot_balls_bowled_percentage 
                FROM bowlingdf 
                WHERE bowlerName = '{player_name}'
                GROUP BY bowlerName;
            """
        }
    }

    role_query = f"""
        SELECT playingRole AS playingRole 
        FROM playersdf 
        WHERE name = '{player_name}';
    """
    
    role_df = mysql(role_query)
    
    if role_df.empty:
        print(f"No role found for player: {player_name}")
        return []

    role = role_df.iloc[0]['playingRole']
    
    print(f"Role retrieved for {player_name}: {role}")

    if role in ['Top Order Batter', 'Middle Order Batter', 'Wicketkeeper Batter', 'Opening Batter', 'Batter']:
        kpi_queries = queries["Batter"]
    elif role in ['Bowler']:
        kpi_queries = queries["Bowler"]
    elif role in ['Allrounder', 'Batting Allrounder', 'Bowling Allrounder']:
        kpi_queries = queries["Allrounder"]
    else:
        print(f"Unrecognized role: {role}")
        return []
    
    kpi_data = []
    for kpi_name, query in kpi_queries.items():
        formatted_query = query.format(player_name=player_name)
        
        kpi_df = mysql(formatted_query)
        if not kpi_df.empty:
            kpi_value = kpi_df.iloc[0].values[0]
            kpi_value = round(kpi_value, 2)
            kpi_data.append({"Metric": kpi_name, "Value": kpi_value})
        else:
            print(f"No KPI data found for {kpi_name} for player: {player_name}")
    
    kpi_df = pd.DataFrame(kpi_data)
    return kpi_df


player_name = 'Rohit Sharma'
kpi_df = generate_player_kpis(player_name)
kpi_df


Role retrieved for Rohit Sharma: Top Order Batter


,Metric,Value
0,Total Runs Scored,116.00
1,Batting Average,19.33
2,Average Balls Faced,18.17
3,Average StrikeRate,89.49
4,Boundary Percentage,13.76
5,Boundaries Hit Per Match,2.50
6,Boundary Frequency (balls per boundary),7.00


### Visualization

In [22]:
import plotly.express as px
import plotly.graph_objects as go
team_colors = {
    'Afghanistan': 'green', 
    'Australia': 'yellow', 
    'Bangladesh': '#006400',  
    'England': '#00247D', 
    'India': '#0099FF',  
    'Ireland': '#009B77',  
    'Namibia': 'gray',  
    'Netherlands': 'darkOrange',  
    'New Zealand': '#000000',  
    'Pakistan': '#004B49',  
    'Scotland': 'mediumpurple',  
    'South Africa': '#008C8C',  
    'Sri Lanka': '#FFD700',  
    'U.A.E.': 'chocolate',  
    'West Indies': '#8A2C2D',  
    'Zimbabwe': 'tomato'  
}


def h_bar_plot(df, x_column, y_column, title, x_title, y_title):
    fig = px.bar(df, x=x_column, y=y_column, color=y_column, text=df[x_column],
                 title=title, labels={x_column: x_title, y_column: y_title},
                 color_discrete_map=team_colors, orientation='h')

    fig.update_traces(marker_line_width=1.5, opacity=0.8, textposition='outside')

    team_options = df[y_column].unique()
    dropdown_buttons = [
        {'label': 'All Teams', 'method': 'update', 'args': [{'visible': [True] * len(df)}]}
    ]

    for team in team_options:
        visibility = [df[y_column][i] == team for i in range(len(df))]
        dropdown_buttons.append({
            'label': team,
            'method': 'update',
            'args': [{'visible': visibility}]
        })

    fig.update_layout(
        title_font=dict(size=24, color='#CC0000', family='Calibri Black'),
        font=dict(family='Calibri'),
        xaxis=dict(title=x_title, title_font=dict(size=18), tickfont=dict(size=16)),
        yaxis=dict(title=y_title, title_font=dict(size=18), tickfont=dict(size=16)),
        showlegend=False,
        plot_bgcolor='ghostwhite',
        paper_bgcolor='ghostwhite',
        updatemenus=[{
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'x': 1.15,  
            'y': 1.15,  
            'xanchor': 'right',  
            'yanchor': 'top',  
            'font': dict(size=18) ,
            'pad': dict(r=10, t=10)
        }]
    )

    fig.show()



def bar_plot(df, x_column, y_column, title, x_title, y_title):
    fig = px.bar(df, x=x_column, y=y_column, color=x_column, text=df[y_column],
                 title=title, labels={x_column: x_title, y_column: y_title},
                 color_discrete_map=team_colors)

    fig.update_traces(marker_line_width=1.5, opacity=0.8, textposition='outside')

    team_options = df[x_column].unique()
    dropdown_buttons = [
        {'label': 'All Teams', 'method': 'update', 'args': [{'visible': [True] * len(df)}]}
    ]

    for team in team_options:
        visibility = [df[x_column][i] == team for i in range(len(df))]
        dropdown_buttons.append({
            'label': team,
            'method': 'update',
            'args': [{'visible': visibility}]
        })
    fig.update_layout(
        title_font=dict(size=28, color='#CC0000', family='Calibri Black'),  
        font=dict(family='Calibri'),
        xaxis=dict(title=x_title, title_font=dict(size=18), tickfont=dict(size=16)),  
        yaxis=dict(title=y_title, title_font=dict(size=18), tickfont=dict(size=16)),  
        showlegend=False,
        plot_bgcolor='ghostwhite',  
        paper_bgcolor='ghostwhite',
        updatemenus=[{
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'x': 1.15,  
            'y': 1.15,  
            'xanchor': 'right',  
            'yanchor': 'top',  
            'font': dict(size=18) ,
            'pad': dict(r=10, t=10)
        }]
    )

    fig.show()


def create_scatter_plot(df, x_column, y_column, title, x_title, y_title, size_column=None):
    fig = go.Figure()
    dropdown_buttons = [{'label': 'All Teams', 'method': 'update', 'args': [{'visible': [True] * len(df['team'].unique())}, {'title': title}]}]
    
    for i, (team, color) in enumerate(team_colors.items()):
        team_data = df[df['team'] == team]
        if not team_data.empty:
            hover_text = team_data.apply(lambda row: f'Team: {team}<br>{x_title}: {row[x_column]:.2f}<br>{y_title}: {row[y_column]:.2f}', axis=1)
            
            fig.add_trace(go.Scatter(
                x=team_data[x_column],
                y=team_data[y_column],
                mode='markers',
                marker=dict(
                    size=18, 
                    color=color,
                    opacity=1,
                    line=dict(width=2, color=color)
                ),
                hovertext=hover_text,
                hoverinfo='text',
                name=team,
                showlegend=True,
                visible=True  
            ))
            visibility = [False] * len(df['team'].unique())
            visibility[i] = True  # Make this team visible when selected
            dropdown_buttons.append({
                'label': team,
                'method': 'update',
                'args': [
                    {'visible': visibility},  # Set the visibility for this team
                    {'title': f'{title}: {team}'}
                ]
            })
    fig.data[0].visible = True
    
    fig.update_layout(
        title=title,
        title_font=dict(size=28, color='#CC0000', family='Calibri Black'),
        xaxis=dict(
            title=x_title,
            title_font=dict(size=18),
            tickfont=dict(size=16),
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=0.5,
            zeroline=False,
            showline=True,
            linewidth=1,
            linecolor='black'
        ),
        yaxis=dict(
            title=y_title,
            title_font=dict(size=18),
            tickfont=dict(size=16),
            showgrid=True,
            gridcolor='lightgray',
            gridwidth=0.5,
            zeroline=False,
            showline=True,
            linewidth=1,
            linecolor='black'
        ),
        plot_bgcolor='ghostwhite',
        paper_bgcolor='ghostwhite',
        showlegend=True,
        updatemenus=[{
            'buttons': dropdown_buttons,
            'direction': 'down',
            'showactive': True,
            'x': 1.25,  
            'y': 1.1
        }]
    )
    fig.show()




### Tournament Analysis


In [46]:
q = """
SELECT
    winner AS team,
    CASE 
        WHEN margin LIKE '%runs%' THEN 'Batting First(Defending)'
        WHEN margin LIKE '%wickets%' THEN 'Bowling First(Chasing)'
    END AS winning_method,
    COUNT(*) AS win_count
FROM matchdf
WHERE margin LIKE '%runs%' OR margin LIKE '%wickets%'
GROUP BY winner, winning_method;
"""
q3 = mysql(q)  

fig = go.Figure()

all_teams_data = q3.groupby('winning_method').sum().reset_index()

fig.add_trace(go.Pie(
    labels=all_teams_data['winning_method'],
    values=all_teams_data['win_count'],
    name='All Teams',
    marker=dict(colors=['#1f77b4', '#2ca02c']),
    textinfo='percent+value',
    insidetextfont=dict(size=18,)
))

for team in q3['team'].unique():
    team_data = q3[q3['team'] == team]
    fig.add_trace(go.Pie(
        labels=team_data['winning_method'],
        values=team_data['win_count'],
        name=team,
        textinfo='percent+value',
        marker=dict(colors=['#1f77b4', '#2ca02c']),
        visible=False  
    ))

dropdown_buttons = [
    {
        'label': 'All Teams',  
        'method': 'update',
        'args': [
            {'visible': [True] + [False] * len(q3['team'].unique())},  
            {'title': 'Winning Strategies: Batting First vs Bowling First'}
        ]
    }
]

for i, team in enumerate(q3['team'].unique()):
    dropdown_buttons.append({
        'label': team,
        'method': 'update',
        'args': [
            {'visible': [False] * (i+1) + [True] + [False] * (len(q3['team'].unique()) - (i + 1))},  
            {'title': f'Winning Strategies:Batting First vs Bowling First ({team})'}
        ]
    })

fig.update_layout(
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True,
    }],
    title="Winning Strategies: Batting First vs Bowling First",
    title_font_size=28,
    title_font=dict(size=28, color="#CC0000", family="calibri Black"),
    width=800,
    height=600,
    legend=dict(font=dict(size=18)),
    paper_bgcolor='ghostwhite'
)

fig.show()


In [24]:
playersdf['bowlingStyle'].unique()

array(['Right Arm Offbreak', 'Right Arm Medium Fast', nan,
       'Slow Left Arm Orthodox', 'Legbreak', 'Right Arm Fast',
       'Left Arm Fast Medium', 'Right Arm Fast Medium',
       'Right Arm Medium', 'Left Arm Fast', 'Legbreak Googly',
       'Right Arm Offbreak, Legbreak Googly', 'Left Arm Medium Fast',
       'Left Arm Wrist Spin', 'Right Arm Medium Fast, Right Arm Offbreak',
       'Left Arm Medium', 'Right Arm Medium, Legbreak',
       'Right Arm Medium, Right Arm Offbreak'], dtype=object)

In [25]:
q = """
SELECT
    p.team AS team,
    CASE 
        WHEN p.bowlingstyle IN ('Right Arm Offbreak', 'Slow Left Arm Orthodox', 'Legbreak', 'Left Arm Wrist Spin', 'Legbreak Googly', 'Right Arm Offbreak, Legbreak Googly', 'Left Arm Wrist Spin', 'Right Arm Medium, Legbreak', 'Right Arm Medium, Right Arm Offbreak') THEN 'Spinner'
        WHEN p.bowlingstyle IN ('Right Arm Medium Fast', 'Right Arm Fast', 'Left Arm Fast Medium', 'Right Arm Fast Medium', 'Right Arm Medium', 'Left Arm Fast', 'Right Arm Medium Fast, Right Arm Offbreak', 'Left Arm Medium Fast', 'Left Arm Medium') THEN 'Fast Bowlers'
        ELSE 'Other'
    END AS bowler_type,
    SUM(b.wickets) AS total_wickets
FROM bowlingdf b
JOIN playersdf p ON b.bowlerName = p.Name
GROUP BY p.team, bowler_type;
"""
q24 = mysql(q)

fig = go.Figure()

all_teams_data = q24.groupby('bowler_type').sum().reset_index()

fig.add_trace(go.Pie(
    labels=all_teams_data['bowler_type'],
    values=all_teams_data['total_wickets'],
    name='All Teams',
    marker=dict(colors=['#1f77b4', '#ff7f0e']),
    textinfo='percent+value',
    insidetextfont=dict(size=18)
))

teams = q24['team'].unique()
for team in teams:
    team_data = q24[q24['team'] == team]
    fig.add_trace(go.Pie(
        labels=team_data['bowler_type'],
        values=team_data['total_wickets'],
        name=team,
        marker=dict(colors=['#1f77b4', '#ff7f0e']),
        textinfo='percent+value',
        insidetextfont=dict(size=18),
        visible=False  
    ))

dropdown_buttons = [
    {
        'label': 'All Teams',
        'method': 'update',
        'args': [
            {'visible': [True] + [False] * len(teams)},
            {'title': 'Wickets by Spinners vs Fast Bowlers '}
        ]
    }
]

for i, team in enumerate(teams):
    dropdown_buttons.append({
        'label': team,
        'method': 'update',
        'args': [
            {'visible': [False] * (i + 1) + [True] + [False] * (len(teams) - (i + 1))},
            {'title': f'Wickets by Spinners vs Fast Bowlers({team})'}
        ]
    })

fig.update_layout(
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True,
        
    }],
    title="Wickets by Spinners vs Fast Bowlers",
    title_font=dict(size=28, color="#CC0000", family="calibri Black"),
    width=800,
    height=600,
    legend=dict(font=dict(size=18)),
    paper_bgcolor='ghostwhite'
)

fig.show()


In [26]:
q = """
WITH BattingCategory AS (
  SELECT
    teamInnings,
    matchId,
    CASE
      WHEN battingPos IN (1, 2) THEN 'Openers'
      WHEN battingPos IN (3, 4, 5) THEN 'Middle Order'
      WHEN battingPos IN (6, 7) THEN 'Lower Middle Order'
      ELSE 'Lower Order'
    END AS "Batting Category",
    runs,
    balls,
    "4s",
    "6s"
  FROM battingdf
),
CategoryMetrics AS (
  SELECT
    teamInnings AS team,
    "Batting Category",
    ROUND(SUM(runs), 2) AS "Total Runs",
    ROUND(SUM(runs) / COUNT(DISTINCT matchId), 2) AS "Average Runs Per Category",
    ROUND(SUM(balls), 2) AS "Total Balls Faced",
    ROUND((SUM("4s") + SUM("6s")) * 100.0 / SUM(balls), 2) AS "Boundary Percentage"
  FROM BattingCategory
  GROUP BY teamInnings, "Batting Category"
),
OverallMetrics AS (
  SELECT
    ROUND(SUM(runs) / COUNT(DISTINCT matchId), 2) AS "Overall Average Runs Per Innings"
  FROM BattingCategory
)
SELECT
  cm.team,
  cm."Batting Category",
  cm."Total Runs",
  cm."Average Runs Per Category" as "Average Runs" ,
  ROUND(cm."Total Runs" * 100.0 / NULLIF(cm."Total Balls Faced", 0), 2) AS "AVG Strike Rate",
  cm."Total Balls Faced",
  cm."Boundary Percentage",
  om."Overall Average Runs Per Innings"
FROM CategoryMetrics cm
JOIN OverallMetrics om ON 1=1;

"""

q5 = mysql(q)
category_colors = {
    'Openers': 'blue',
    'Middle Order': 'green',
    'Lower Middle Order': 'orange',
    'Lower Order': 'red'
}
fig = go.Figure()

overall_data = q5.groupby("Batting Category").agg({
    "Average Runs": "mean",
    "AVG Strike Rate": "mean",
    "Total Runs": "sum"
}).reset_index()

max_bubble_size = 60
min_bubble_size = 10
sizeref_overall = 2.0 * max(overall_data["Total Runs"]) / (max_bubble_size**2)

fig = go.Figure()

fig.add_trace(go.Scatter(
    x=overall_data["Average Runs"],
    y=overall_data["AVG Strike Rate"],
    mode='markers+text',  
    marker=dict(
        size=overall_data["Total Runs"],
        sizemode='area',
        sizeref=sizeref_overall,
        sizemin=min_bubble_size,
        color=[category_colors[cat] for cat in overall_data["Batting Category"]],
        opacity=0.8,
        line=dict(width=2, color='black')
    ),
    text=overall_data["Batting Category"],  
    textposition='bottom center',  
    hovertext=overall_data.apply(lambda row: f'Batting Category: {row["Batting Category"]}<br>Average Runs: {row["Average Runs"]:.2f}<br>AVG Strike Rate: {row["AVG Strike Rate"]:.2f}', axis=1),
    hoverinfo='text',
    name='All Teams',
    visible=True
))

for team in q5['team'].unique():
    team_data = q5[q5['team'] == team]

    if not team_data.empty:
        sizeref_team = 2.0 * max(team_data["Total Runs"]) / (max_bubble_size**2)

        fig.add_trace(go.Scatter(
            x=team_data["Average Runs"],
            y=team_data["AVG Strike Rate"],
            mode='markers+text',  
            marker=dict(
                size=team_data["Total Runs"],
                sizemode='area',
                sizeref=sizeref_team,
                sizemin=min_bubble_size,
                color=[category_colors[cat] for cat in team_data["Batting Category"]],  # Corrected this line
                opacity=0.8,
                line=dict(width=2, color='black')
            ),
            text=team_data["Batting Category"],  
            textposition='bottom center',  
            hovertext=team_data.apply(lambda row: f'Team: {team}<br>Batting Category: {row["Batting Category"]}<br>Average Runs: {row["Average Runs"]:.2f}<br>AVG Strike Rate: {row["AVG Strike Rate"]:.2f}', axis=1),
            hoverinfo='text',
            name=team,
            visible=False
        ))

dropdown_buttons = [
    {'label': 'All Teams', 'method': 'update', 'args': [{'visible': [True] + [False] * (len(fig.data) - 1)}, {'title': 'Batting Category Performance'}]}
]

for team in q5['team'].unique():
    visibility = [False] * len(fig.data)
    visibility[0] = False  
    for i, trace in enumerate(fig.data):
        if trace.name == team:
            visibility[i] = True
    dropdown_buttons.append({
        'label': team,
        'method': 'update',
        'args': [
            {'visible': visibility},
            {'title': f'Batting Category Performance: {team}'}
        ]
    })

fig.update_layout(
    title="Batting Category Performance",
    title_font=dict(size=28, color='#CC0000', family='Calibri Black'),
    xaxis_title="Average Runs",
    yaxis_title="Average Strike Rate",
    font=dict(size=18, family='Calibri'),
    plot_bgcolor='ghostwhite',
    paper_bgcolor='ghostwhite',
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        zeroline=False,
        showline=True,
        linecolor='black',
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        zeroline=False,
        showline=True,
        linecolor='black',
    ),
    updatemenus=[{
        'buttons': dropdown_buttons,
        'direction': 'down',
        'showactive': True,
        'x': 1.15,
        'y': 1.15,
        'font': dict(size=18)
    }],
    showlegend=False,
    autosize=True,  
)

fig.show()

In [27]:
q="""SELECT
    team,
    SUM(CASE WHEN result = 'Win' THEN count ELSE 0 END) AS Wins,
    SUM(CASE WHEN result = 'Loss' THEN count ELSE 0 END) AS Losses,
    SUM(CASE WHEN result = 'No Result' THEN count ELSE 0 END) AS "No Results"
FROM (
    SELECT team1 AS team,
           CASE
               WHEN team1 = winner THEN 'Win'
               WHEN team2 = winner THEN 'Loss'
               ELSE 'No Result'
           END AS result,
           COUNT(*) AS count
    FROM matchdf
    GROUP BY team1, result

    UNION ALL

    SELECT team2 AS team,
           CASE
               WHEN team2 = winner THEN 'Win'
               WHEN team1 = winner THEN 'Loss'
               ELSE 'No Result'
           END AS result,
           COUNT(*) AS count
    FROM matchdf
    GROUP BY team2, result
) AS results
GROUP BY team;"""

q20=mysql(q)
df_melted = q20.melt(id_vars='team', var_name='Result', value_name='Count')


fig = px.bar(
    df_melted,
    x='team',
    y='Count',
    color='Result',
    color_discrete_map={
        'Wins': '#06D001',  
        'Losses': '#FF0000',  
        'No Results': 'grey'  
    },
    title='Match Outcome Breakdown by Team: Wins, Losses, and No Results',
    labels={'Count': 'Number of Matches'},
    text='Count',  
    height=500,  
)


fig.update_layout(
    barmode='stack',  
    xaxis_title='Team',  
    yaxis_title='Number of Matches',  
    legend_title='Match Result', 
    title_font=dict(size=28, color='#CC0000', family='Calibri Black'),  
    font=dict(size=18, family='Calibri'), 
    plot_bgcolor='ghostwhite',  
    paper_bgcolor='ghostwhite',  
)

fig.update_xaxes(showgrid=False)
fig.update_yaxes(showgrid=True)

fig.show()



In [28]:
q="""
SELECT 
    ground, 
    winner, 
    COUNT(*) AS win_count
FROM 
    matchdf
GROUP BY 
    ground, 
    winner
ORDER BY 
    ground;


"""
q26=mysql(q)
q26

ground_performance_matrix = q26.pivot(index='ground', columns='winner', values='win_count').fillna(0)

fig = go.Figure(data=go.Heatmap(
                   z=ground_performance_matrix.values,
                   x=ground_performance_matrix.columns,
                   y=ground_performance_matrix.index,
                  colorscale = [
                               [0.0, 'white'],       
                               [0.2, '#CCFFFF'],     
                               [0.4, '#99CCCC'],     
                               [0.6, '#66B2B2'],     
                               [0.8, '#339999'],    
                               [1.0, '#006666']],
                   hoverongaps=False,
                   text=ground_performance_matrix.values,
                   texttemplate="%{text}",
                   showscale=True))

fig.update_layout(
    title='Team Performance Across Grounds',
    title_font=dict(size=28, color='#CC0000', family='Calibri Black'),  
    xaxis_title='Team',
    yaxis_title='Ground',
    xaxis_title_font=dict(size=18,  family='Calibri'),  
    yaxis_title_font=dict(size=18,family='Calibri'),  
    font=dict(size=16, family='Calibri'),  
    plot_bgcolor='ghostwhite',  
    paper_bgcolor='ghostwhite',  
    xaxis_nticks=36,
    yaxis_nticks=36
)

fig.show()


### Teams Analysis

In [29]:
q = """
SELECT winner AS team,
       COUNT(*) AS win_count
FROM matchdf
WHERE winner IS NOT NULL AND winner != 'No Result'
GROUP BY team
ORDER BY win_count DESC;
"""

q4 = mysql(q)


h_bar_plot(q4, 'win_count', 'team', 'Number of Wins by Team', 'Number of Wins', 'Team')



In [30]:
q= """SELECT team,ROUND(SUM(wins) * 1.0 / SUM(total_matches) * 100 ,2) AS "Win Loss Ratio"
      FROM(
      SELECT team1 AS team,
                   SUM(CASE WHEN winner = team1 THEN 1 ELSE 0 END) AS wins,
                   COUNT(*) AS total_matches
            FROM matchdf
            GROUP BY team1
            UNION ALL
            SELECT team2 AS team,
                   SUM(CASE WHEN winner = team2 THEN 1 ELSE 0 END) AS wins,
                   COUNT(*) AS total_matches
            FROM matchdf
            GROUP BY team2
        ) AS combined
        GROUP BY team
        ORDER BY "Win Loss Ratio" DESC;
    """

q8=mysql(q)

h_bar_plot(q8, 'Win Loss Ratio', 'team', 'Wins Loss Ratio of Each Team(Percentage)', 'Win/Loss Ratio ', 'Team')




In [31]:
q="""SELECT teamInnings as Team, SUM(runs) AS total_runs
FROM battingdf
GROUP BY Team
ORDER BY total_runs DESC;
"""
q6=mysql(q)
bar_plot(q6, 'Team', 'total_runs', 'Total Runs Scored by Each Team', 'Team', 'Total Runs')


In [32]:

q = """
SELECT teamInnings as Team, ROUND(SUM(runs)/COUNT(DISTINCT matchId),2) AS "Avg Runs Per Match"
from battingdf
GROUP BY Team
ORDER BY "Avg Runs Per Match" DESC;
"""

q7 = mysql(q)

bar_plot(q7, 'Team', 'Avg Runs Per Match', 'Average Runs Scored Per Match by Each Team', 'Team', 'Average Runs')



In [33]:
q= """
SELECT 
    bowlingTeam AS team,
    SUM(wickets) / COUNT(DISTINCT matchId) AS "Average Wickets Per Match"
FROM 
    bowlingdf
GROUP BY 
    bowlingTeam
ORDER BY 
    "Average Wickets Per Match" DESC;
"""
q9=mysql(q)
h_bar_plot(q9, 'Average Wickets Per Match', 'team', 'Average Wickets Per Match by Each Team', 'Average Wickets  ', 'Team')



In [34]:

q="""SELECT
    teamInnings AS team,
    ROUND(SUM(runs)/COUNT (DISTINCT matchId)) AS "Average Runs",
    SUM(balls) AS total_balls_faced,
    (SUM(runs) * 100.0 / NULLIF(SUM(balls), 0)) AS strike_rate
FROM battingdf
WHERE battingPos IN (1, 2) 
GROUP BY team;

"""
q15=mysql(q)

create_scatter_plot(q15, 'Average Runs', 'strike_rate', 'Openers Average Runs vs Strike Rate by Team(Batting Position 1&2)', 'Average Runs', 'Strike Rate')



    

In [35]:

q="""SELECT
    teamInnings AS team,
    ROUND(SUM(runs)/COUNT (DISTINCT matchId)) AS "Average Runs",
    SUM(balls) AS total_balls_faced,
    (SUM(runs) * 100.0 / NULLIF(SUM(balls), 0)) AS strike_rate
FROM battingdf
WHERE battingPos IN (3,4,5) 
GROUP BY team;

"""
q16=mysql(q)

create_scatter_plot(q16, 'Average Runs', 'strike_rate', 'Middle Order Average Runs vs Strike Rate by Team(Batting Position 3,4,5)', 'Average Runs', 'Strike Rate')



In [36]:
q="""SELECT
    teamInnings AS team,
    ROUND(SUM(runs)/COUNT (Distinct matchId)) AS "Average Runs",
    SUM(balls) AS total_balls_faced,
    (SUM(runs) * 100.0 / NULLIF(SUM(balls), 0)) AS strike_rate
FROM battingdf
WHERE battingPos IN (6,7) 
GROUP BY team;
"""
q17=mysql(q)
create_scatter_plot(q17, 'Average Runs', 'strike_rate','Lower Middle Order Average Runs vs Strike Rate by Team(Batting Position 6,7)', 'Average Runs', 'Strike Rate')


### Batting Analysis

In [37]:
q="""
SELECT teamInnings AS Team,
ROUND((SUM(runs) * 1.0 / SUM(balls)) * 100,2) AS "Strike Rate"
FROM battingdf
GROUP BY Team
ORDER BY  "Strike Rate" DESC;
"""
q11=mysql(q)
bar_plot(q11, 'Team', 'Strike Rate', 'Average Batting Strike Rate by Each Team', 'Team', 'Batting Strike Rate')



In [38]:
q="""WITH BattingDetails AS (
    SELECT
        b.teamInnings AS batting_team,
        bs.matchId,
        bs."0s",
        bs.total_balls_bowled,
        bs.wides,
        bs.noBalls
    FROM
        battingdf b
    JOIN
        bowlingdf bs
    ON
        b.matchId = bs.matchId
        AND b.teamInnings = CASE 
            WHEN bs.bowlingTeam = m.team1 THEN m.team2
            WHEN bs.bowlingTeam = m.team2 THEN m.team1
        END
    JOIN
        matchdf m
    ON
        bs.matchId = m.matchId
)
SELECT 
    b.batting_team as Team,
    SUM(b."0s") AS total_dot_balls_faced,
    SUM(b.total_balls_bowled - b.wides - b.noBalls) AS total_balls_faced,
    ROUND((SUM(b."0s") * 100.0) / SUM(b.total_balls_bowled - b.wides - b.noBalls),2) AS "Dot Ball Percentage"
FROM BattingDetails b
GROUP BY Team
ORDER BY "Dot Ball Percentage" DESC;


    """
q13=mysql(q)
h_bar_plot(q13, 'Dot Ball Percentage', 'Team','Dot Ball Percentage by Each Team(Batting)', 'Team', 'Dot Ball Percentage ')




In [39]:
q = """
SELECT b.batsmanName as BatsmanName, MAX(b.runs) AS "Top Individual Scores", DATE(m.matchDate) as MatchDate,
    m.ground as Ground,
    CASE 
        WHEN teamInnings = m.team1 THEN m.team2
        WHEN teamInnings = m.team2 THEN m.team1
    END AS VS
FROM battingdf b
JOIN matchdf m ON b.matchId = m.matchId
GROUP BY b.batsmanName, m.matchId, m.matchDate, m.ground, VS
ORDER BY "Top Individual Scores" DESC
LIMIT 10;
"""

q1 = mysql(q)

q1['Batsman vs Team'] = q1['BatsmanName'] + ' vs ' + q1['VS']

fig = go.Figure(data=[go.Table(
    header=dict(
        values=["Batsman ", "Runs", "Match Date", "Ground"],
        fill_color='lightgrey',
        align='left',
        height=40
    ),
    cells=dict(
        values=[q1['Batsman vs Team'], q1['Top Individual Scores'], q1['MatchDate'], q1['Ground']],
        fill_color='white',
        align='left',
        height=40
    )
)])

fig.update_layout(
    title='Top 10 Individual Scores',
    title_font=dict(size=28, color='#CC0000', family='Calibri Black'),
    font=dict(size=14, color='black', family='Calibri'),
    plot_bgcolor='ghostwhite',
    paper_bgcolor='ghostwhite',
    margin=dict(l=20, r=20, t=50, b=20)
)

fig.show()


In [40]:
q="""SELECT batsmanName as PlayerName,
    ROUND(CAST(SUM(runs) AS FLOAT) / NULLIF(COUNT(DISTINCT CASE WHEN `out/not_out` = 1 THEN matchId END), 0) ,2)AS "Average Runs",
    SUM(balls) AS total_balls_faced,
    ROUND((SUM(runs) * 100.0 / NULLIF(SUM(balls), 0)),2)AS strike_rate
FROM battingdf
GROUP BY PlayerName
HAVING SUM(balls) >= 75 
ORDER BY "Average Runs" DESC
LIMIT 10;

"""
q18=mysql(q)


q="""SELECT batsmanName as PlayerName,
    ROUND(CAST(SUM(runs) AS FLOAT) / NULLIF(COUNT(DISTINCT CASE WHEN `out/not_out` = 1 THEN matchId END), 0) ,2)AS "Average Runs",
    SUM(balls) AS total_balls_faced,
    ROUND((SUM(runs) * 100.0 / NULLIF(SUM(balls), 0)),2)AS strike_rate
FROM battingdf
GROUP BY PlayerName
HAVING SUM(balls) >= 75 
ORDER BY "Average Runs" DESC
LIMIT 10;

"""
q18=mysql(q)


fig = px.scatter(q18, 
                 x='Average Runs', 
                 y='strike_rate',  
                 size='Average Runs',    
                 color='PlayerName',  
                 title='Top 10 Batters: AVG Runs vs AVG SR (Minimum 75 balls Faced)',
                 labels={'Average Runs': 'Average Runs', 'strike_rate': 'Strike Rate'})

fig.update_traces(marker=dict(opacity=1))

fig.update_layout(
    title={
        'text': 'Top 10 Batters: Average Runs vs Strike Rate<span style="font-size: 18px">(*Minimum 75 balls Faced)</span>',
        'font': dict(size=28, color='#CC0000', family='Calibri Black'),
    },
    xaxis_title='Average Runs',
    yaxis_title='Strike Rate',
    font=dict(size=18, family='Calibri'),
    plot_bgcolor='ghostwhite',
    paper_bgcolor='ghostwhite',
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.5,
        showline=True,
        linewidth=1,
        linecolor='black'
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.5,
        showline=True,
        linewidth=1,
        linecolor='black'
    )
)

fig.show()

### Bowling Analysis

In [41]:
q= """SELECT bowlingTeam AS Team,
      ROUND(SUM(runs) * 1.0 / SUM(overs),2) AS "Economy Rate"
      FROM bowlingdf
      GROUP BY Team
      ORDER BY  "Economy Rate" DESC;
"""
q10=mysql(q)
h_bar_plot(q10,'Economy Rate','Team', 'Average Economy Rate by Each Team', 'Team', 'Economy Rate')

In [42]:
q= """
SELECT bowlingTeam AS Team,
ROUND((SUM(total_balls_bowled) * 1.0 / SUM(wickets)),2) AS "Bowling Strike Rate"
FROM bowlingdf
GROUP BY Team
ORDER BY "Bowling Strike Rate" DESC;
"""
q12=mysql(q)
bar_plot(q12, 'Team','Bowling Strike Rate', 'Average Bowling Strike Rate by Each Team', 'Team', 'Bowling Strike Rate')



In [43]:
q="""
SELECT bowlingTeam AS Team,
SUM(extras) / COUNT(DISTINCT matchId) AS "Avg Extras Per Match"
FROM bowlingdf
GROUP BY bowlingTeam
ORDER BY "Avg Extras Per Match" DESC;
"""
q14=mysql(q)
bar_plot(q14, 'Team','Avg Extras Per Match', 'Average Extras Conceded Per Match by Each Team', 'Team', 'Extras Conceded')





In [44]:
q = """
SELECT b.bowlerName AS "Bowler Name",
       MAX(b.wickets) AS "Top Individual Wickets",
       b.runs AS "Runs Given",
       b.economy AS Economy,
       DATE(m.matchDate) AS "MatchDate",
       m.ground AS "Ground",
       CASE 
           WHEN b.bowlingTeam = m.team1 THEN m.team2
           WHEN b.bowlingTeam = m.team2 THEN m.team1
           ELSE NULL
       END AS "VS"
FROM bowlingdf b
JOIN matchdf m ON b.matchId = m.matchId
GROUP BY b.bowlerName, m.matchId, DATE(m.matchDate), m.ground, "VS"
ORDER BY "Top Individual Wickets" DESC
LIMIT 10;
"""

q2 = mysql(q)


q2['Bowler vs Team'] = q2['Bowler Name'] + ' vs ' + q2['VS']

fig = go.Figure(data=[go.Table(
    header=dict(
        values=["Bowler ", "Wickets", "Runs Given","Economy Rate", "Match Date", "Ground"],
        fill_color='lightgrey',
        align='left',
        height=40
    ),
    cells=dict(
        values=[q2['Bowler vs Team'], q2['Top Individual Wickets'], q2['Runs Given'],q2["Economy"], q2['MatchDate'], q2['Ground']],
        fill_color='white',
        align='left',
        height=40
    )
)])

fig.update_layout(
    title='Top 10 Individual Wickets ',
    title_font=dict(size=28, color='#CC0000', family='Calibri Black'),
    font=dict(size=14,color="Black", family='Calibri'),
    plot_bgcolor='ghostwhite',
    paper_bgcolor='ghostwhite',
    margin=dict(l=20, r=20, t=50, b=20)
)

fig.show()

In [45]:
q="""SELECT
    BowlerName AS PlayerName,
    SUM(wickets) AS total_wickets,
    SUM(runs) AS total_runs_conceded,
    SUM(total_balls_bowled) AS total_balls_bowled,
    ROUND(CAST(SUM(runs) AS FLOAT) / NULLIF(SUM(total_balls_bowled)/ 6, 0),2) AS economy_rate
FROM bowlingdf
GROUP BY PlayerName
HAVING SUM(total_balls_bowled) >= 50 
ORDER BY total_wickets DESC
LIMIT 10;

"""
kpi=mysql(q)


fig = px.scatter(kpi, 
                 x='total_wickets', 
                 y='economy_rate',  
                 size='total_wickets',    
                 color='PlayerName',      
                 title='Top 10 Bowlers: Total Wickets vs Economy Rate',
                 labels={'total_wickets': 'Total Wickets', 'economy_rate': 'Economy Rate'})

fig.update_traces(marker=dict(opacity=1))

fig.update_layout(
    title={
        'text': 'Top 10 Bowlers: Total Wickets vs Economy Rate<span style="font-size: 16px">(*Minimum 50 balls bowled)</span>',
        'font': dict(size=28, color='#CC0000', family='Calibri Black'),
    },
    xaxis_title='Total Wickets',
    yaxis_title='Economy Rate',
    font=dict(size=18, family='Calibri'),
    plot_bgcolor='ghostwhite',
    paper_bgcolor='ghostwhite',
    xaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.5,
        showline=True,
        linewidth=1,
        linecolor='black'
    ),
    yaxis=dict(
        showgrid=True,
        gridcolor='lightgray',
        gridwidth=0.5,
        showline=True,
        linewidth=1,
        linecolor='black'
    )
)

fig.show()